In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./Data/restaurant_data')
target_df = pd.read_csv('./Data/target_data')

restaurants_df = df.merge(target_df,
                         how = 'inner',
                         left_on = ['Unnamed: 0'],
                         right_on = ['Unnamed: 0']
                         )
restaurants_df.drop('Unnamed: 0' , axis=1,inplace =True)

In [3]:
restaurants_df

,user_id,prod_id,rating,date,review,name,target
0,22558,859,5.0,2007-05-28,Looking for some authentic Japanese food at re...,Yakitori Totto,1
1,78837,915,4.0,2009-10-05,Pepe Rosso is where you go when you're in SOHO...,Pepe Rosso To Go,1
2,19590,668,5.0,2013-09-17,I had waited to return a couple other times to...,Eataly NYC,1
3,7175,287,3.0,2013-10-30,This place is always busy - partly because it'...,Heidi’s House,1
4,137409,744,5.0,2014-12-04,Love this place! I am not a regular yelper I d...,Bodhi Tree,0
...,...,...,...,...,...,...,...
285989,2708,72,4.0,2013-06-23,Noodle taste great. Its nice and chewy. Im suc...,Totto Ramen,1
285990,115028,570,4.0,2014-12-11,Despite frequently visiting and previously liv...,Up Thai,1
285991,2018,281,4.0,2013-09-23,Definitely a place to come for a good dinner o...,Kofte Piyaz,1
285992,19933,839,4.0,2012-05-26,"At first, I didn't know if I enjoyed this food...",Great NY Noodle Town,1


In [4]:
corpus2 = restaurants_df[['review','target']]

In [5]:
corpus2

,review,target
0,Looking for some authentic Japanese food at re...,1
1,Pepe Rosso is where you go when you're in SOHO...,1
2,I had waited to return a couple other times to...,1
3,This place is always busy - partly because it'...,1
4,Love this place! I am not a regular yelper I d...,0
...,...,...
285989,Noodle taste great. Its nice and chewy. Im suc...,1
285990,Despite frequently visiting and previously liv...,1
285991,Definitely a place to come for a good dinner o...,1
285992,"At first, I didn't know if I enjoyed this food...",1


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
from nltk.corpus import stopwords

In [8]:
sw = stopwords.words('english')

In [9]:
sw += ['food','place','good',"great"]

In [10]:
body = corpus2.review
target = corpus2.target

In [11]:
X_train, X_test, y_train, y_test = train_test_split(body,
                                                    target,
                                                    random_state = 42,
                                                    stratify = target)

In [12]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, string
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models, optimizers

In [14]:
encoder = preprocessing.LabelEncoder()

In [15]:
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [16]:
count_vect = CountVectorizer(analyzer='word',stop_words = sw)
count_vect.fit(X_train)

all_words_dict = (count_vect.vocabulary_)

Xtrain_count =  count_vect.transform(X_train)
#Xtest_count =  count_vect.transform(X_test)

In [17]:
type(all_words_dict)

dict

In [18]:
X_test_count = count_vect.transform(X_test)

In [38]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=5000,lowercase=False)
tfidf_vect.fit(X_train)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xtest_tfidf =  tfidf_vect.transform(X_test)

In [44]:
#tf_train = tfidf_vect.fit_transform(X_train, y_train)

In [37]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', ngram_range=(2,3), max_features=5000,lowercase =False)
xtrain_tfidf_ngram  = tfidf_vect_ngram.fit_transform(X_train)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

In [ ]:
# %%time
# CPU times: user 5min 6s, sys: 7.66 s, total: 5min 14s
# Wall time: 5min 18s


# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3), max_features=5000, lowercase = False)
xtrain_tfidf_ngram_chars = tfidf_vect_ngram_chars.fit_transform(X_train)
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test) 

In [22]:
def train_model(classifier, feature_vector_train = Xtrain_count, label=y_train, feature_vector_valid = X_test_count, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, y_test)

In [23]:
# nb_classifier = MultinomialNB()

In [24]:
# b_train_score = accuracy_score(target, nb_train_preds)
# nb_test_score = accuracy_score(newsgroups_test.target, nb_test_preds)

In [25]:
# vectorizer = TfidfVectorizer()

In [26]:
# tf_idf_data_test = vectorizer.fit_transform(X_train)

In [27]:
#nb_classifier.fit(tf_idf_data_test, y_train)

# nb_train_preds = nb_classifier.predict(Xtrain_count)
# nb_test_preds = nb_classifier.predict(X_test_count)

In [28]:
train_model(naive_bayes.MultinomialNB())

0.8699702093735576

In [46]:
nb = naive_bayes.MultinomialNB()

In [51]:
nb.fit(Xtrain_count,y_train)
nb.predict(X_train)

ValueError: could not convert string to float: 'Had a great experience here. \xa0Food was outstanding. Love the open kitchen. Wine list was fantastic. \xa0Bar tender was great (let me try the mead before ordering a glass - they have a mead selection). Nice vibe to the place. \xa0Service was just how I like it: Polite and unobtrusive, not detracting from the evening at all. \xa0I will be coming back.'

In [29]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, y_train, xtest_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

NB, WordLevel TF-IDF:  0.8982223527601785


In [30]:
# Naive Bayes on Ngram Level TF IDF Vectors
accuracy2 = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy2)

NB, N-Gram Vectors:  0.8982223527601785


In [31]:
# Naive Bayes on Character Level TF IDF Vectors
accuracy3 = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy3)

NB, CharLevel Vectors:  0.8982223527601785


In [32]:
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print "NB, WordLevel TF-IDF: ", accuracy

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("NB, WordLevel TF-IDF: ", accuracy)? (<ipython-input-32-ff24c64ffa4e>, line 2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
count_vect = CountVectorizer()
X_train_count = count_vect.fit_transform(body)

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)

In [ ]:

xvalid_count =  count_vect.transform(valid_x)